In [ ]:
import torch
import cv2
import numpy as np
import os

In [ ]:
name=input("please enter the name of image with extension")
IMAGE_PATH = "Input_images"
IMAGE_PATH =os.path.join(IMAGE_PATH,name)

In [ ]:


#model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best1.pt')  
im = IMAGE_PATH 
results = model(im)

In [ ]:
x=results.xyxy[0][0][0]
y=results.xyxy[0][0][1]
mx=results.xyxy[0][0][2]
my=results.xyxy[0][0][3]


In [ ]:
def grabcut_algorithm(original_image, bounding_box):
    
    segment = np.zeros(original_image.shape[:2],np.uint8)
    
    x,y,width,height = bounding_box
    segment[y:y+height, x:x+width] = 1

    background_mdl = np.zeros((1,65), np.float64)
    foreground_mdl = np.zeros((1,65), np.float64)
    
    cv2.grabCut(original_image, segment, bounding_box, background_mdl, foreground_mdl, 5,
    cv2.GC_INIT_WITH_RECT)

    new_mask = np.where((segment==2)|(segment==0),0,1).astype('uint8')

    original_image = original_image*new_mask[:,:,np.newaxis]
    
    return original_image

In [ ]:
oimage=cv2.imread(IMAGE_PATH)
# oimage=cv2.resize(oimage,(600,600))
x=round(x.item())
y=round(y.item())
mx=round(mx.item())
my=round(my.item())
image=grabcut_algorithm(oimage,(x-20,y-20,mx-x+20,my-y+20))

In [ ]:
ratiox=0
ratioy=0
img=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(img, 15, 255, cv2.THRESH_BINARY)
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
area_treshold = 1000
for c in cnts:
    if cv2.contourArea(c) > area_treshold :
#       x,y,w,h = cv2.boundingRect(c)
#       cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)
        x=cv2.minAreaRect(c)
        b=cv2.boxPoints(x)
        b=np.int0(b)
        cv2.polylines(oimage,[b],True,(0,255,0),2)
        ratiox=8.6/max(x[1][1],x[1][0])
        ratioy=5.4/min(x[1][1],x[1][0])

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt') 


im = IMAGE_PATH 


results = model(im)


In [ ]:
x=results.xyxy[0][0][0]
y=results.xyxy[0][0][1]
mx=results.xyxy[0][0][2]
my=results.xyxy[0][0][3]


In [ ]:
x=round(x.item())
y=round(y.item())
mx=round(mx.item())
my=round(my.item())
image=grabcut_algorithm(oimage,(x-50,y,mx-x+50,my-y+50))

In [ ]:
img=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(img, 15, 255, cv2.THRESH_BINARY)
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
area_treshold = 1000
for c in cnts:
    if cv2.contourArea(c) > area_treshold :
#       x,y,w,h = cv2.boundingRect(c)
#       cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)
        x=cv2.minAreaRect(c)
        b=cv2.boxPoints(x)
        b=np.int0(b)
        cv2.polylines(oimage,[b],True,(0,255,0),2)
        cv2.putText(oimage, f"Height : {round(x[1][0]*ratiox,1)} CM  Width : {round(x[1][1]*ratioy,1)} CM", (int(x[0][0]-x[1][1]/2), int(x[0][1]-x[1][0]/2) + 30), cv2.FONT_HERSHEY_PLAIN ,2, (0,255,0),3)
cv2.imshow("Output",oimage)
OUTPUT_PATH=os.path.join("Results","marked"+name)
cv2.imwrite(OUTPUT_PATH,oimage)
cv2.waitKey(0)